L2-Regularization Method on Link Prediction Task of Cora Dataset
--------------------------

### All libraries we need

In [1]:


import os
import os.path as osp
import time
import copy
import psutil
import itertools
import tracemalloc
import gc
import statistics as stat
import sys
import argparse
import numpy as np
import random
import matplotlib.pyplot as plt
import scipy.sparse as sp


import torch
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import SplineConv
from torch_geometric.typing import WITH_TORCH_SPLINE_CONV
from torch.optim import Adam

import torch.nn.utils.prune as prune

import warnings
warnings.filterwarnings('ignore')
os.environ["DGLBACKEND"] = "pytorch"

### Regularization Rate
#### Regularization rates range from the following numbers:

In [2]:

0, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2,0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1e2, 1e3, 1e6
;

''

### Functions for Mmeasuring criterias

In [3]:
def get_num_parameters(model: nn.Module, count_nonzero_only=False) -> int:
    """
    calculate the total number of parameters of model
    :param count_nonzero_only: only count nonzero weights
    """
    num_counted_elements = 0
    for param in model.parameters():
        if count_nonzero_only:
            num_counted_elements += param.count_nonzero()
        else:
            num_counted_elements += param.numel()
    return num_counted_elements

# Function to get CPU usage
def get_cpu_usage():
    return psutil.cpu_percent(interval=1)



# Function to approximate power consumption (Assume some average power usage per CPU percentage point)
def estimate_power_usage(cpu_usage):
    base_power_usage = 10  # Assumed base power usage in watts
    power_per_percent = 0.5  # Assumed additional watts per CPU usage percent
    return base_power_usage + (power_per_percent * cpu_usage)

# The model size based on the number of parameters
def get_model_size_param(model: nn.Module, data_width=32, count_nonzero_only=False) -> int:
    """
    calculate the model size in bits
    :param data_width: #bits per element
    :param count_nonzero_only: only count nonzero weights
    """
    return get_num_parameters(model, count_nonzero_only) * data_width

Byte = 8
KiB = 1024 * Byte
MiB = 1024 * KiB
GiB = 1024 * MiB


### Clearing the arguments

In [4]:


sys.argv = ['']
parser = argparse.ArgumentParser()
parser.add_argument('--model_name',type=str,default='Spline')
parser.add_argument('--dataset_name',type=str,default='Cora')
parser.add_argument('--num_deg',type=int,default=1000)
parser.add_argument('--num_layers', type=int, default=2)
parser.add_argument('--hidden_units',type=int,default=16)
parser.add_argument('--max_epoch',type=int,default=50)
parser.add_argument('--max_cycle',type=int,default=10)
parser.add_argument('--weight_decay',type=float,default=0)
parser.add_argument('--lr',type=float,default=0.001)
###############################################################


args = parser.parse_args()
print(args)

Namespace(model_name='Spline', dataset_name='Cora', num_deg=1000, num_layers=2, hidden_units=16, max_epoch=50, max_cycle=10, weight_decay=0, lr=0.001)


### save path for model

In [5]:

if not os.path.isdir('checkpoint'):
    os.mkdir('checkpoint')
if not os.path.isdir(os.path.join('checkpoint', f"Cora")):
    os.mkdir(os.path.join('checkpoint', f"Cora"))
ckpt_dir = f"./checkpoint/Cora/"



def save_best(ckpt_dir, epoch, state, model_name, eval_acc, is_best, is_reg):
    print('saving....')
    model.to(device)
    state_save = {
        'net':state,
        'epoch':epoch,
        'acc': eval_acc 
        }
    best_pth_name = f'{args.model_name}_best.pth'
    reg_pth_name = f'{args.model_name}_reg_best.pth'
    
  
    if is_reg & is_best:
        ckpt_path = os.path.join(ckpt_dir, reg_pth_name) 
        torch.save(state_save, ckpt_path)
    
     
    if is_reg == False & is_best:
        ckpt_path = os.path.join(ckpt_dir, best_pth_name)  
        torch.save(state_save, ckpt_path)
           
        
    model.to(device)

## Loading Dataset

In [6]:

if not WITH_TORCH_SPLINE_CONV:
    quit("This example requires 'torch-spline-conv'")

dataset = 'Cora'
transform = T.Compose([
    T.RandomNodeSplit(num_val=500, num_test=500),
    T.TargetIndegree(),
])
path =  'data'
dataset = Planetoid(path, dataset, transform=transform)
data = dataset[0]


# Model

In [7]:


class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = SplineConv(dataset.num_features, 16, dim=1, kernel_size=2)
        self.conv2 = SplineConv(16, dataset.num_classes, dim=1, kernel_size=2)

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        x = F.dropout(x, training=self.training)
        x = F.elu(self.conv1(x, edge_index, edge_attr))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_attr)
        return F.log_softmax(x, dim=1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-3)

In [8]:
def train(l2):
    
    model.train()
    optimizer.zero_grad()
    loss=F.nll_loss(model(data)[data.train_mask], data.y[data.train_mask])
    l2_reg = torch.tensor(0.)
    for param in model.parameters():
        l2_reg += torch.norm(param)
        
    # Combine the loss function with L2 regularization
    loss += (l2 * l2_reg)
    loss.backward() 
    
    optimizer.step()
    return loss
          


def evaluate_Spline(model, mask):
    model.eval()
    for _, mask in mask:
        log_probs, accs = model(data), []
        # calculate acc
        pred = log_probs[mask].max(1)[1]
        acc=pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        # Calculate loss
        loss=F.nll_loss(model(data)[mask], data.y[mask])
    
    eval_state = {'loss': loss / (mask.sum().item()), 'acc': acc}
    return eval_state



@torch.no_grad()
def test(model):
    model.eval()
    log_probs, accs = model(data), []
    for _, mask in data('train_mask', 'test_mask'):
        pred = log_probs[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

### Implement base model and regularized model

#### Setting Regularization rate

In [26]:
### Regularization Rate
l2_lambda = 0.9
#The number of epochs 
num_epochs=100
# The number of iteration
num_iterations=10

In [16]:
# This is a dictionary to save all measurements. Aftre measuring, we can compute mean and std of each item.
Eva_final=dict()

# The following are all list of criteria for measurements. 
# We collect all desired datas of each list across iterations. 
# Then, we compute average and std of each list.


#Base model
Base_model_accuracy=[]
T_base_model=[]
Num_parm_base_model=[]
Base_model_size=[]
Base_Energy_Consumption=[]
Base_Cpu_Usage=[]
Base_Memory_Usage=[]

#regularized model
Reg_model_accuracy=[]
T_Reg_model=[]
Num_parm_Reg_model=[]
Reg_model_size=[]
Reg_Energy_Consumption=[]
Reg_Cpu_Usage=[]
Reg_Memory_Usage=[]

# Here is the dictionary to record the list of all measurements
Eva_measure={'base model accuracy':Base_model_accuracy,
            'time inference of base model':T_base_model,
            'number parmameters of base model':Num_parm_base_model,
            'base model size':Base_model_size,
            'energy consumption of base model':Base_Energy_Consumption,
            'cpu usage of base model':Base_Cpu_Usage,
            'memory usage of base model':Base_Memory_Usage,
            'regularized model accuracy': Reg_model_accuracy,
            'time inference of regularized model':T_Reg_model,
            'number parmameters of regularized model':Num_parm_Reg_model,
            'regularized model size':Reg_model_size,
            'energy consumption of regularized model':Reg_Energy_Consumption,
            'cpu usage of regularized model':Reg_Cpu_Usage,
            'memory usage of regularized model':Reg_Memory_Usage
            }


### Training and Applying Regularization

In [20]:

for i in range(num_iterations):
        print('________________________________________________')
        print('************************************************')
        print(f"This is iteration :{i+1}")
        
        print(f'Training and evaluation before regularization ')
        print("Starting training...")
        Eva=dict()# It is a dictionary to arrange output of this iteration
        
        best_acc=0  
        early_stopping=num_epochs*0.2


        #Training base model
        print(f'Training and evaluation before pruning ')
        
        model = Net().to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-3)
        best_acc=0  
        save_epoch=100
        is_reg=False
        reg_rate=0
        
        print(f" The rate of regularization is :{reg_rate}")
        for epoch in range(num_epochs):  
            acc=[]
            loss_list = []
            
            train(reg_rate)        

            eval_state =  evaluate_Spline(model,data('val_mask'))
            acc_eval=eval_state['acc']
            # report train msg
            if epoch % 5 == 0:   
                print(f"Eval Epoch: {epoch} | Loss: {eval_state['loss']:.3f} | Acc: {eval_state['acc']:.3f}")


            # only save the best model
            if eval_state['acc'] > best_acc:
                early_stop_count = 0
            else:
                early_stop_count += 1

            if early_stop_count > early_stopping:
                break
            is_best = (eval_state['acc'] > best_acc)
            if is_best:
                best_acc = eval_state['acc']
                early_stop_count = 0

            if is_best or epoch % save_epoch == 0:
                 save_best(ckpt_dir, epoch, model.state_dict(), args.model_name, eval_state['acc'], is_best, is_reg)   

        #### load the best model
        base_model_path = os.path.join(ckpt_dir, f'{args.model_name}_best.pth') 
        checkpoint = torch.load(base_model_path)
        model.load_state_dict(checkpoint['net'])
        recover_model = lambda:model.load_state_dict(best_checkpoint['state_dict'])

        # Start monitoring CPU and memory usage, model size, number of parametes, time inference and  power consumption
        gc.collect()
        time.sleep(5)  # Add a 5-second delay to stabilize the initial state
        tracemalloc.start()  # Start tracking memory allocations
        snapshot_before = tracemalloc.take_snapshot()#take a snapshot of the current memory state before starting the measurement.

        t0 = time.perf_counter()
        initial_cpu_usage = get_cpu_usage()
        power_usage = estimate_power_usage(initial_cpu_usage)

        train_acc, base_model_accuracy=test(model)
    

        base_cpu_usage = get_cpu_usage()
        t1 = time.perf_counter()
        t_base_model=t1-t0

        snapshot_after = tracemalloc.take_snapshot()
        tracemalloc.stop()
        top_stats = snapshot_after.compare_to(snapshot_before, 'lineno')

        base_total_memory_diff = sum([stat.size_diff for stat in top_stats])
        base_energy_consumption = power_usage * t_base_model
        base_model_size = os.path.getsize(base_model_path)
        num_parm_base_model=get_num_parameters(model, count_nonzero_only=True)

        gc.collect()
        time.sleep(5) 
        
        print(f'*****Results of base model*********')

        print(f"base model has accuracy on test set={base_model_accuracy:.2f}%")
        print(f"base model has size={base_model_size:.2f} bit")
        print(f"The time inference of base model is ={t_base_model}") 
        print(f"The number of parametrs of base model is:{num_parm_base_model}") 

        print(f"Energy Consumption : {base_energy_consumption:.3f}")
        print(f"total memory usage of base model':{base_total_memory_diff} ")
        print(f"cpu usage of base model':{base_cpu_usage:.3f} %")


        #Update Eva dictionary
        Eva.update({'base model accuracy': base_model_accuracy,
                    'time inference of base model': t_base_model,
                    'number parmameters of base model': num_parm_base_model,
                    'size of base model': base_model_size, 
                    'energy consumption of base model':base_energy_consumption,
                    'total memory usage of base model':base_total_memory_diff,
                    'cpu usage of base model':base_cpu_usage
                   })

        gc.collect()
        time.sleep(5) 
        
        #### Regularization of the Model
    
        best_reg_checkpoint = dict()
        best_reg_acc = 0
        is_reg=True
        reg_rate=l2_lambda
        print(f" The rate of regularization is :{reg_rate}")
       
        
        model = Net().to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-3)
        for epoch in range(num_epochs):
            # At the end of each train iteration, we have to apply the pruning mask
            #    to keep the model sparse during the training
            acc=[]
            loss_list = []
            train(reg_rate)        

            eval_state =  evaluate_Spline(model,data('val_mask'))
            acc_eval=eval_state['acc']
            # report train msg
            if epoch % 20 == 0:   
               print(f"Eval Epoch: {epoch} | Loss: {eval_state['loss']:.3f} | Acc: {eval_state['acc']:.3f}")
            

            # only save the best model
            if eval_state['acc'] > best_reg_acc :
                early_stop_count = 0
            else:
                early_stop_count += 1

            if early_stop_count >early_stopping:
                break
            is_best = (eval_state['acc'] > best_reg_acc )
           
            if is_best:
                best_acc = eval_state['acc']
                early_stop_count = 0
               
          
           
            if is_best or epoch % save_epoch == 0:
                
                save_best(ckpt_dir, epoch, model.state_dict(), args.model_name, eval_state['acc'], is_best, is_reg)


        #### load the best regularized model
        reg_pth_name = f'{args.model_name}_reg_best.pth'
        reg_model_path = os.path.join(ckpt_dir, reg_pth_name) 
        checkpoint = torch.load(reg_model_path)
        model.load_state_dict(checkpoint['net'])
        recover_model = lambda:model.load_state_dict(checkpoint['net'])
     
       

        # Result of regularization
      

        gc.collect()
        time.sleep(5)  
        tracemalloc.start() 
        snapshot_before = tracemalloc.take_snapshot()

        t0 = time.perf_counter()
        initial_cpu_usage = get_cpu_usage()
        power_usage = estimate_power_usage(initial_cpu_usage)

     
        train_acc, regularized_model_accuracy=test(model)

        regularized_cpu_usage = get_cpu_usage()
        t1 = time.perf_counter()
        t_regularized_model=t1-t0

        snapshot_after = tracemalloc.take_snapshot()
        tracemalloc.stop()
        top_stats = snapshot_after.compare_to(snapshot_before, 'lineno')

        regularized_total_memory_diff = sum([stat.size_diff for stat in top_stats])
        regularized_energy_consumption = power_usage * t_regularized_model
        regularized_model_size = os.path.getsize( reg_model_path )
        num_parm_regularized_model=get_num_parameters(model, count_nonzero_only=True)

        gc.collect()
        time.sleep(5)  # Add a 5-second delay to stabilize the initial state    
        
 
        
        print('****************Result of regularized model ******************')
    
        
        print(f"{l2_lambda} regularized model has accuracy on test set={regularized_model_accuracy:.2f}%")
        print(f"{l2_lambda} regularized model has size={regularized_model_size:.2f} bit")
        print(f"The time inference of {l2_lambda} regularized model is ={t_regularized_model}") 
        print(f"The number of parametrs of {l2_lambda} regularized model is:{num_parm_regularized_model}") 

        print(f"Energy Consumption of {l2_lambda} regularized model: {regularized_energy_consumption:.3f}")
        print(f"total memory usage of {l2_lambda} regularized model':{regularized_total_memory_diff} ")
        print(f"cpu usage of {l2_lambda} regularized model':{regularized_cpu_usage:.3f} %")


        #Update Eva dictionary
        Eva.update({'regularized model accuracy': regularized_model_accuracy,
                    'time inference of regularized model': t_regularized_model,
                    'number parmameters of regularized model': num_parm_regularized_model,
                    'size of regularized model': regularized_model_size, 
                    'energy consumption of regularized model':regularized_energy_consumption,
                    'total memory usage of regularized model':regularized_total_memory_diff,
                    'cpu usage of regularized model':regularized_cpu_usage
                   })

        gc.collect()
        time.sleep(5)   

        

        Base_model_accuracy.append(Eva['base model accuracy'])
        T_base_model.append(Eva['time inference of base model'])
        Num_parm_base_model.append(int(Eva['number parmameters of base model']))
        Base_model_size.append(int(Eva['size of base model']))
        Base_Energy_Consumption.append(Eva['energy consumption of base model'])
        Base_Cpu_Usage.append(Eva['cpu usage of base model'])
        Base_Memory_Usage.append(Eva['total memory usage of base model'])

        Reg_model_accuracy.append(Eva['regularized model accuracy'])
        T_Reg_model.append(Eva['time inference of regularized model'])
        Num_parm_Reg_model.append(int(Eva['number parmameters of regularized model']))
        Reg_model_size.append(int(Eva['size of regularized model']))
        Reg_Energy_Consumption.append(Eva['energy consumption of regularized model'])
        Reg_Cpu_Usage.append(Eva['cpu usage of regularized model'])
        Reg_Memory_Usage.append(Eva['total memory usage of regularized model'])




________________________________________________
************************************************
Training and evaluation before regularization 
Starting training...
Training and evaluation before pruning 
 The rate of regularization is :0
Eval Epoch: 0 | Loss: 0.004 | Acc: 0.366
saving....
saving....
saving....
saving....
saving....
Eval Epoch: 5 | Loss: 0.002 | Acc: 0.716
saving....
saving....
saving....
saving....
saving....
Eval Epoch: 10 | Loss: 0.001 | Acc: 0.836
saving....
saving....
saving....
saving....
Eval Epoch: 15 | Loss: 0.001 | Acc: 0.862
saving....
saving....
saving....
saving....
*****Results of base model*********
base model has accuracy on test set=0.88%
base model has size=279991.00 bit
The time inference of base model is =19.680765499942936
The number of parametrs of base model is:69143
Energy Consumption : 281.435
total memory usage of base model':7536 
cpu usage of base model':8.000 %
 The rate of regularization is :0.9
Eval Epoch: 0 | Loss: 0.004 | Acc: 0.350
sa

### Computing Mean and Std of Lists

In [21]:
Eva_final=dict()
base_model_accuracy_mean = stat.mean(Base_model_accuracy)
base_model_accuracy_std =  stat.stdev(Base_model_accuracy)
Eva_final.update({'Ave of base model accuracy':float(format(base_model_accuracy_mean, '.3f'))})
Eva_final.update({'Std of base model accuracy':float(format(base_model_accuracy_std, '.3f'))})
base_model_accuracy = "{:.3f} ± {:.3f}".format(base_model_accuracy_mean ,base_model_accuracy_std)
print(f"Base model accuracy is:{base_model_accuracy}")

                 
t_base_model_mean =stat.mean(T_base_model)
t_base_model_std =stat.stdev(T_base_model)  
Eva_final.update({'Ave of time inference of base model':float(format(t_base_model_mean, '.3f'))})
Eva_final.update({'Std of time inference of base model':float(format(t_base_model_std, '.3f'))})
t_base_model = "{:.3f} ± {:.3f}".format(t_base_model_mean ,t_base_model_std)
print(f"Time inference of Base model :{t_base_model}")


num_parm_base_model_mean = stat.mean(Num_parm_base_model)
num_parm_base_model_std = stat.stdev(Num_parm_base_model)
Eva_final.update({'Ave of number parmameters of base model':num_parm_base_model_mean})
Eva_final.update({'Std of number parmameters of base model':num_parm_base_model_std})
num_parm_base_model = "{:.3f} ± {:.3f}".format(num_parm_base_model_mean ,num_parm_base_model_std)
print(f"Time number of parameters of Base model :{num_parm_base_model}")

base_model_size_mean = stat.mean(Base_model_size)
base_model_size_std = stat.stdev(Base_model_size)
Eva_final.update({'Ave of base model size':base_model_size_mean})
Eva_final.update({'Std of base model size':base_model_size_std})
base_model_size_model = "{:.3f} ± {:.3f}".format(base_model_size_mean ,base_model_size_std)
print(f"The size of Base model :{base_model_size} bytes")


base_energy_consumption_mean = stat.mean(Base_Energy_Consumption)
base_energy_consumption_std = stat.stdev(Base_Energy_Consumption)
Eva_final.update({'Ave of energy consumption of base model':base_energy_consumption_mean })
Eva_final.update({'Std of energy consumption of base model':base_energy_consumption_std})
base_energy_consumption = "{:.3f} ± {:.3f}".format(base_energy_consumption_mean ,base_energy_consumption_std)
print(f"The energy consumption of Base model :{base_energy_consumption} ")


base_cpu_usage_mean = stat.mean(Base_Cpu_Usage)
base_cpu_usage_std = stat.stdev(Base_Cpu_Usage)
Eva_final.update({'Ave of cpu usage of base model':base_cpu_usage_mean})
Eva_final.update({'Std of cpu usage of base model':base_cpu_usage_std})
base_cpu_usage = "{:.3f} ± {:.3f}".format(base_cpu_usage_mean ,base_cpu_usage_std)
print(f"The CPU usage of Base model :{base_cpu_usage} ")


base_memory_usage_mean = stat.mean(Base_Memory_Usage)
base_memory_usage_std = stat.stdev(Base_Memory_Usage)
Eva_final.update({'Ave of memory usage of base model':base_memory_usage_mean})
Eva_final.update({'Std of memory usage of base model':base_memory_usage_std})
base_memory_usage = "{:.3f} ± {:.3f}".format(base_memory_usage_mean ,base_memory_usage_std)
print(f"The memory usage of Base model :{base_memory_usage} ")

print(100 * "=")
####################################################

reg_model_accuracy_mean =stat.mean(Reg_model_accuracy)
reg_model_accuracy_std = stat.stdev(Reg_model_accuracy)
Eva_final.update({'Ave of regularized model accuracy':float(format(reg_model_accuracy_mean, '.3f'))})
Eva_final.update({'Std of regularized model accuracy':float(format(reg_model_accuracy_std, '.3f'))})
reg_model_accuracy = "{:.3f} ± {:.3f}".format(reg_model_accuracy_mean ,reg_model_accuracy_std)
print(f"Regularized model accuracy is:{reg_model_accuracy}")
                 

t_reg_model_mean = stat.mean(T_Reg_model)
t_reg_model_std =stat.stdev(T_Reg_model)
Eva_final.update({'Ave of time inference of regularized model':float(format(t_reg_model_mean, '.3f'))})
Eva_final.update({'Std of time inference of regularized model':float(format(t_reg_model_std, '.3f'))})
t_reg_model = "{:.3f} ± {:.3f}".format(t_reg_model_mean ,t_reg_model_std)
print(f"Time inference of Regularized model :{t_reg_model}")

num_parm_reg_model_mean = stat.mean(Num_parm_Reg_model)
num_parm_reg_model_std = stat.stdev(Num_parm_Reg_model)
Eva_final.update({'Ave of number parmameters of regularized model':num_parm_reg_model_mean})
Eva_final.update({'Std of number parmameters of regularized model':num_parm_reg_model_std})
num_parm_reg_model = "{:.3f} ± {:.3f}".format(num_parm_reg_model_mean ,num_parm_reg_model_std)
print(f"Time number of parameters of Regularized model :{num_parm_reg_model}")

reg_model_size_mean =stat.mean( Reg_model_size)
reg_model_size_std = stat.stdev(Reg_model_size)
Eva_final.update({'Ave of regularized model size':reg_model_size_mean})
Eva_final.update({'Std of regularized model size':reg_model_size_std })
reg_model_size = "{:.3f} ± {:.3f}".format(reg_model_size_mean ,reg_model_size_std)
print(f"The size of Regularized model :{reg_model_size} bytes")

reg_energy_consumption_mean = stat.mean(Reg_Energy_Consumption)
reg_energy_consumption_std = stat.stdev(Reg_Energy_Consumption)
Eva_final.update({'Ave of energy consumption of regularized model':reg_energy_consumption_mean })
Eva_final.update({'Std of energy consumption of regularized model':reg_energy_consumption_std})
reg_energy_consumption = "{:.3f} ± {:.3f}".format(reg_energy_consumption_mean ,reg_energy_consumption_std)
print(f"The energy consumption of Regularized model :{reg_energy_consumption} ")


reg_cpu_usage_mean = stat.mean(Reg_Cpu_Usage)
reg_cpu_usage_std = stat.stdev(Reg_Cpu_Usage)
Eva_final.update({'Ave of cpu usage of regularized model':reg_cpu_usage_mean})
Eva_final.update({'Std of cpu usage of regularized model':reg_cpu_usage_std})
reg_cpu_usage = "{:.3f} ± {:.3f}".format(reg_cpu_usage_mean ,reg_cpu_usage_std)
print(f"The CPU usage of Regularized model :{reg_cpu_usage} ")


reg_memory_usage_mean = stat.mean(Reg_Memory_Usage)
reg_memory_usage_std = stat.stdev(Reg_Memory_Usage)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of memory usage of regularized model':reg_memory_usage_mean})
Eva_final.update({'Std of memory usage of regularized model':reg_memory_usage_std})
reg_memory_usage = "{:.3f} ± {:.3f}".format(reg_memory_usage_mean ,reg_memory_usage_std)
print(f"The memory usage of Regularized model :{reg_memory_usage} ")



#################################


print(f"All measurement about regularization process of rate:{l2_lambda} ")   
Eva_final

Base model accuracy is:0.887 ± 0.005
Time inference of Base model :19.121 ± 0.494
Time number of parameters of Base model :69143.000 ± 0.000
The size of Base model :279991 bytes
The energy consumption of Base model :212.368 ± 46.159 
The CPU usage of Base model :5.650 ± 6.318 
The memory usage of Base model :7442.000 ± 108.983 
Regularized model accuracy is:0.551 ± 0.390
Time inference of Regularized model :18.708 ± 0.180
Time number of parameters of Regularized model :69143.000 ± 0.000
The size of Regularized model :280039.000 ± 0.000 bytes
The energy consumption of Regularized model :194.407 ± 16.063 
The CPU usage of Regularized model :0.300 ± 0.383 
The memory usage of Regularized model :6384.000 ± 1126.988 
All measurement about regularization process of rate:0.9 


{'Ave of base model accuracy': 0.887,
 'Std of base model accuracy': 0.005,
 'Ave of time inference of base model': 19.121,
 'Std of time inference of base model': 0.494,
 'Ave of number parmameters of base model': 69143,
 'Std of number parmameters of base model': 0.0,
 'Ave of base model size': 279991,
 'Std of base model size': 0.0,
 'Ave of energy consumption of base model': 212.3677599121438,
 'Std of energy consumption of base model': 46.15872447553138,
 'Ave of cpu usage of base model': 5.65,
 'Std of cpu usage of base model': 6.318491381123609,
 'Ave of memory usage of base model': 7442,
 'Std of memory usage of base model': 108.98317913023703,
 'Ave of regularized model accuracy': 0.551,
 'Std of regularized model accuracy': 0.39,
 'Ave of time inference of regularized model': 18.708,
 'Std of time inference of regularized model': 0.18,
 'Ave of number parmameters of regularized model': 69143,
 'Std of number parmameters of regularized model': 0.0,
 'Ave of regularized model s

### Recording results in txt file

In [25]:


dataset_name = 'Cora'
Pruning_Method='Regularization'
max_epoch = 100
resume = True
result_folder ='pathresult/'
if not os.path.exists(result_folder):
    os.makedirs(result_folder)


file_name = result_folder+Pruning_Method+'_'+'with rate of regularization of'+'_'+str(l2_lambda)+'_on_'+dataset_name+'_'+'Node task'+'_'+str(max_epoch)+'.txt'


with open(file_name, 'w') as f:
        f.write('%s:%s\n'%('dataset_name', 'Cora'))
        f.write('%s:%s\n'%('max_epoch', max_epoch))
        f.write('%s:%s\n'%('sparsity', l2_lambda))
        for key, value in Eva_final.items():
            f.write('%s:%s\n'%(key, value))
        for key, value in Eva_measure.items():
            f.write('%s:%s\n' % (key, ','.join(map(str, value)))) 